#### The first two cells are for interpolating missing data.
#### If you already have the data, do not run them. They take ~5-6 minutes.

The first cell runs through each feature in the joined data, interpolating missing values
when 5 or less are present sequentially.
This interpolation is done through linear and cubic splines.
Note that this does not remove the large gaps of null values

The second cell interpolates all the large null gaps for some features, each to a different level of success.
The list of interpolated features is named 'desiredColumns'.


In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d

df = pd.read_csv("Joined Influent and Rainfall and Weather and Groundwater and Creek Gauge.csv", parse_dates=["DateTime"])
df["SWTP Total Influent Flow"] = np.array([np.nan if x < 3.7 else x for x in df["SWTP Total Influent Flow"]])

for col in df.columns:
    # getting an array from the dataframe
    index = 0
    arr = np.array(df[col])

    # looping through each index in the array to find nulls
    while index < len(arr):
        # a null has been found
        if np.isnan(arr[index]):
            # getting width of null gap
            width = 1
            while index + width < len(arr) and np.isnan(arr[index + width]):    #not reach the end and is still null
                width += 1
            # print("Index = {i}, width = {w}".format(i = index, w = width))

            if width < 6 and index + width + 1 < len(arr):
                # interpolate data!
                # want next 10 values before and after if not null
                # else however many there are available until first null, guarenteed at least 1
                if width == 1:
                    # linear interpolate
                    x = [index-1] + [index+1]
                    y = [arr[i] for i in x]
                    linInterplator = interp1d(x, y)
                    
                    # adding imputed value into array
                    arr[index] = linInterplator(index)
                
                elif width == 2:
                    # linear interpolate
                    x = [index-1] + [index+2]
                    y = [arr[i] for i in x]
                    linInterplator = interp1d(x, y)

                    # adding imputed values into array
                    arr[index] = linInterplator(index)
                    arr[index+1] = linInterplator(index+1)

                else:
                    # cubic spline interpolation -- sometimes prone to unbelieveable imputed data, but generally close enough to seem right
                    lenForwards = 0
                    while lenForwards < 10 and not np.isnan(arr[index + width + lenForwards + 1]):
                        lenForwards += 1

                    lenBackwards = 0
                    while lenBackwards < 10 and not np.isnan(arr[index - lenBackwards - 1]):
                        lenBackwards += 1
                    
                    # getting x and y values for cubic spline and building spline
                    nullRange = list(range(index, index + width, 1))
                    totalRange = list(range(index - lenBackwards, index + width + lenForwards + 1, 1))
                    x = [x for x in totalRange if x not in nullRange]
                    y = [arr[i] for i in x]
                    cspline = CubicSpline(x, y)

                    # replacing null values in array with interpolated values
                    interpolationRange = list(range(index - 1, index + width + 1, 1))
                    for i in interpolationRange:
                        arr[i] = cspline(i)

            # move index forward past gap, continue searching
            index += width

        # no null gap, so continue searching
        else:
            index += 1
    
    # replacing arr in df with arr with interpolated values
    df[col] = arr

# adding year, month, day, and hour columns
df["Year"] = df["DateTime"].dt.year
df["Month"] = df["DateTime"].dt.month
df["Week Day"] = df["DateTime"].dt.dayofweek
df["Hour"] = df["DateTime"].dt.hour

# saving new interpolated data
df.to_csv("Small Gap Imputed Data.csv", index=False)

In [ ]:
import pandas as pd
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

desiredColumns = ["SWTP Total Influent Flow", "James Gauge Height (ft)", "Wilsons Gauge Height (ft)",
                  "Springfield Plateau Aquifer Depth to Water Level (ft)", "Ozark Aquifer Depth to Water Level (ft)",
                  "HourlyStationPressure", "Fire 168 Hour Rainfall Aggregate", "AT&T 168 Hour Rainfall Aggregate",
                   "Field 168 Hour Rainfall Aggregate", "Bingham 168 Hour Rainfall Aggregate",
                   "Hour", "Month", "Year", "Week Day"]
df = pd.read_csv("Small Gap Imputed Data.csv", usecols=desiredColumns)
data = df.to_numpy()
imputer = MissForest(criterion="squared_error", n_estimators=80)
data_imputed = imputer.fit_transform(data)
imputed_df = pd.DataFrame(data_imputed, columns=df.columns)
dates = pd.read_csv("Small Gap Imputed Data.csv", usecols=["DateTime"])
imputed_df["DateTime"] = np.array(dates["DateTime"])
imputed_df.to_csv("All Gap Imputed Data.csv", index=False)

##### Change the feature variable in the following and run the cell to see interpolated values
##### The SWTP Total Influent Flow looks solid, but the Aquifer features do not look to be interpolated well
##### We should try to interpolate those in a different way, I believe

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

def makeNullRects(dates, y):
    '''This function returns a list of matplotlib.patches.Rectangles where
    np.nan values are present in the y array. If values are consecutive,
    the rectangles will widen as needed.
    Note that this function is made for a figure with an x-axis of dates
    Input:
        dates: x axis date time values
        y: y axis range values as np.array, contains np.nan values

    Returns:
        list of matplotlib.patches.Rectangles located where
        y has np.nan values.

    Rectangle Parameters in function:
        opacityCoeff: how solid rectangles appear
        longRectColor: the color of the rectangles with >=7 width
        shortRectColor: the color of the rectanges with <7 width
    '''
    # setting up rectangle parameters
    opacityCoeff = 0.5
    longRectColor = "red"
    shortRectColor = "magenta"

    # prep work for creating rectangles for nan values
    index = 0
    yMax = np.nanmax(y)
    yMin = np.nanmin(y)
    rectHeight = yMax - yMin
    yRectCoor = yMin
    allRects = []   # this is what will be returned

    # creating rectangle patches
    while index < len(y):

        # if nan exists, then need to create a rectangle patch
        if np.isnan(y[index]):
            xRectCoorIndex = index - 1

            # condition for if first y value is nan
            if index == 0:
                xRectCoorIndex += 1
            
            # condition for if last y value is nan, assumes y is not len 2
            elif index + 1 == len(y):
                xRectCoor = mdates.date2num(dates[xRectCoorIndex])
                coords = (xRectCoor, yRectCoor)
                width = mdates.date2num(dates[xRectCoorIndex + 1]) - mdates.date2num(dates[xRectCoorIndex])
                allRects.append(mpatches.Rectangle(coords, width, rectHeight, color=shortRectColor, alpha=opacityCoeff))
                break
                
            # all other cases
            xRectCoor = mdates.date2num(dates[xRectCoorIndex])

            # checking finding how long the rectangle needs to be--how many consecutive null values
            index += 1
            while np.isnan(y[index]):
                index += 1
            rightEdgeIndex = mdates.date2num(dates[index])

            # making rectangle
            coords = (xRectCoor, yRectCoor)
            width = rightEdgeIndex - xRectCoor
            color = shortRectColor
            if index - xRectCoorIndex > 5:
                color = longRectColor
            allRects.append(mpatches.Rectangle(coords, width, rectHeight, color=color, alpha=opacityCoeff))

        else:
            index += 1

    return allRects

def visualizeMissingValues(dates, arr, fig, ax):
    '''This function plots an array of values with datetime x axis values onto
    a given axis, showing patches of null values if present.
    '''
    ax.plot(dates, arr)

    rects = makeNullRects(dates, arr)
    for rect in rects:
        ax.add_patch(rect)

    formatter = mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), formats=["%Y", "%Y-%b", "%b-%d", "%d %H:%M", "%d %H:%M", "%H:%M"])
    locator = mdates.AutoDateLocator()
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_locator(locator)

    fig.autofmt_xdate()
    return fig, ax

# getting data
originalDf = pd.read_csv("Joined Influent and Rainfall and Weather and Groundwater and Creek Gauge.csv", parse_dates=["DateTime"])
imputedDf = pd.read_csv("All Gap Imputed Data.csv", parse_dates=["DateTime"])


# feature to visualize

feature = "SWTP Total Influent Flow"
# feature = "Ozark Aquifer Depth to Water Level (ft)"
# feature = "Springfield Plateau Aquifer Depth to Water Level (ft)"
# feature = "James Gauge Height (ft)"
# feature = "Wilsons Gauge Height (ft)"
# feature = "Fire 168 Hour Rainfall Aggregate"

# plotting original data and missing gaps
arr = np.array(originalDf[feature])
fig, ax = plt.subplots()
fig, ax = visualizeMissingValues(originalDf["DateTime"], arr, fig, ax)

# plotting interpolated data
imputedArr = np.array(imputedDf[feature])
index = 0
dates = np.array(imputedDf["DateTime"])
while index < len(arr):                                 # looping through arr since it has the null values
    if np.isnan(arr[index]):
        # getting the width of the null area
        lenForward = 0
        while np.isnan(arr[index + lenForward]):
            lenForward += 1

        # domain to plot is [index-1, index+lenforward+1]
        domain = list(range(index-1, index+lenForward+2))
        datesToPlot = [dates[i] for i in domain]
        pointsToPlot = [imputedArr[i] for i in domain]
        ax.plot(datesToPlot, pointsToPlot, "g--")       # green dashed line

        # moving index forward past null gap
        index += lenForward
    else:
        index += 1

plt.show()